# Lecture 10 – Grouping with Subgroups, Merge

## DSC 10, Fall 2021

### Announcements

- Homework 3 due **Saturday 10/16 at 11:59pm.**
- Lab 4 out soon, due **Thursday 10/21 at 11:59pm**.
- Project 1 due **Saturday 10/30 at 11:59pm**.
    - You can work with a partner on your team! 👯‍♀️ See the [Pair Programming guidelines](https://dsc10.com/pair-programming/) on the course website.
    - Use [this sheet](https://docs.google.com/spreadsheets/d/19tnetCMN1z9Kl57kKLtqNzT5ppjub1nDUniNjA-iXKE/edit?usp=sharing) to find a partner.

### Agenda

- Grouping with subgroups.
- Merging.

In [ ]:
import numpy as np
import babypandas as bpd

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')


from IPython.display import display, IFrame

def merging_animation():
    src="https://docs.google.com/presentation/d/e/2PACX-1vSk2FfJ4K_An_CQwcN_Yu5unpJckOZjVQDFqZ78ZTTMmowUsCQKKVnum0_m6TaiGquQ44E3FiS9g2Y4/embed?start=false&loop=false&delayms=60000"
    width=825
    height=500
    display(IFrame(src, width, height))

## Grouping with subgroups

### Our familiar NBA dataset 🏀

In [ ]:
nba = bpd.read_csv('data/nba_salaries.csv').set_index('PLAYER')
nba = nba.assign(SALARY=nba.get("'15-'16 SALARY")).drop(columns="'15-'16 SALARY")
nba

### How big is each team?

- We know how to do this: `.groupby()` with `.count()`.
- **Notice**: team names become the row labels.

In [ ]:
nba.groupby('TEAM').count()

### How much does each team pay in payroll? 💵

- Instead of counting, we want to sum the `'SALARY'` column for each team.

In [ ]:
nba.groupby('TEAM').sum()

### How many players at each position does each team have?

- It seems like grouping would be helpful here, but currently we only know how to compute the size of a group.
- Here we want to count the size of groups within groups. 
    - Specifically, sizes of position groups within team groups.
- There's a way!

In [ ]:
nba.groupby(['TEAM', 'POSITION']).count()

### `.groupby()` with subgroups

- To make groups within groups, pass a list of column names to `.groupby()`:

```py
df.groupby([col_1, col_2, col_3])
```
- Groups by `col_1` first.
- Within each group, groups by `col_2`, and so on.
- The resulting DataFrame has one row per unique combination of entries in the specified columns.

### Notice the index... 🤔

- This is called a "[MultiIndex](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html)".
- We won't worry about the details of MultiIndexes.
- We can use `.reset_index()` to "flatten" our DataFrame back to normal.

In [ ]:
nba.groupby(['TEAM', 'POSITION']).count().reset_index()

### Does order matter?

In [ ]:
nba.groupby(['POSITION', 'TEAM']).count()

**Answer:** Kind of. The order of the indexes will be different, but the resulting DataFrame will contain the same information.

### Which team has the most centers?

In [ ]:
position_counts = nba.groupby(['TEAM', 'POSITION']).count().reset_index()
position_counts

In [ ]:
position_counts[position_counts.get('POSITION') == 'C'] \
.sort_values('SALARY', ascending=False)

In this case, we could've done this without subgroups, too:

In [ ]:
nba[nba.get('POSITION') == 'C'].groupby('TEAM').count().sort_values('POSITION', ascending=False)

### Example: Sea temperatures 🌊

- This dataset contains the sea surface temperature in La Jolla, on many days ranging from August 22, 1916 to May 31, 2019.

In [ ]:
sea_temp = bpd.read_csv('data/sea_temp.csv')
sea_temp

### Discussion Question: Our dataset contains information on 1,234 different months. How do we find the month with the highest average temperature?

What kind of grouping should we do here?

- A) `sea_temp.groupby('SURFACE_TEMP')`
- B) `sea_temp.groupby('MONTH')`
- C) `sea_temp.groupby(['YEAR', 'MONTH'])`
- D) `sea_temp.groupby(['MONTH', 'DAY'])`
- E) `sea_temp.groupby(['MONTH', 'SURFACE_TEMP'])`

### To answer, go to [menti.com](https://menti.com) and enter the code 7374 7077.

In [ ]:
...

### Plot of yearly average surface temperature 📈

In [ ]:
sea_temp.groupby('YEAR') \
        .mean().plot(kind='line', y='SURFACE_TEMP');

### Summary: `.groupby` with subgroups

- Pass a list of columns to make subgroups.
- **Always** use `.reset_index()` after to move the MultiIndex back to the columns.

## Merge 🚗

In [ ]:
phones = bpd.DataFrame().assign(
    Model=['iPhone 13', 'iPhone 13 Pro Max', 'Samsung Galaxy Z Flip', 'Pixel 5a'],
    Price=[799, 1099, 999, 449],
    Screen=[6.1, 6.7, 6.7, 6.3]
)

inventory = bpd.DataFrame().assign(
    Handset=['iPhone 13 Pro Max', 'iPhone 13', 'Pixel 5a', 'iPhone 13'],
    Units=[50, 40, 10, 100],
    Store=['Westfield UTC', 'Westfield UTC', 'Fashion Valley', 'Downtown']
)

In [ ]:
# Phones on the market right now
phones

In [ ]:
# Which phones my stores have in stock in the area
inventory

**Question:** If I sell all of the phones in my inventory, how much will I make in revenue?

### If I sell all of the phones in my inventory, how much will I make in revenue?

In [ ]:
phones.merge(inventory, left_on='Model', right_on='Handset')

### What just happened!? 🤯

In [ ]:
# Click through the presentation that appears
merging_animation()

### `.merge` method

- Pick a "left" and "right" DataFrame.
- Choose a column from each to "merge on".
```python
left_df.merge(
    right_df, 
    left_on=left_column_name,
    right_on=right_column_name
)
```
- `left_on` and `right_on` should be column names (they don't have to be the same).
- One row for every match.
- Deletes rows that don't match!

### If I sell all of the phones in my inventory, how much will I make in revenue?

In [ ]:
phones_merged = phones.merge(inventory, left_on='Model', right_on='Handset')
phones_merged

In [ ]:
(phones_merged.get('Price') * phones_merged.get('Units')).sum()

### Shortcut if column names are the same: `on`

In [ ]:
phones

In [ ]:
inventory_relabeled = inventory.assign(Model=inventory.get('Handset')).drop(columns=['Handset'])
inventory_relabeled

In [ ]:
phones.merge(inventory_relabeled, on='Model')

**Notice**: There's only one column containing phone names now.

### Does order matter? 🤔

In [ ]:
phones

In [ ]:
inventory

In [ ]:
inventory.merge(phones, left_on='Handset', right_on='Model')

**Answer:** The order of the rows and columns will be different, but the content will be the same.

### What if we want to "merge on" an index?

- Instead of using `left_on` or `right_on`, use `left_index=True` or `right_index=True`.

In [ ]:
phones

In [ ]:
inventory_by_handset = inventory.set_index('Handset')
inventory_by_handset

In [ ]:
phones.merge(inventory_by_handset, left_on='Model', right_index=True)

### Discussion Question Setup

In [ ]:
schools = bpd.DataFrame().assign(
    name=['UCSD', 'University of Chicago', 'University of San Diego','Johns Hopkins University', 'UT Austin', 'SDSU', 'UCLA'], 
    city=['La Jolla', 'Chicago', 'San Diego', 'Baltimore', 'Austin', 'San Diego', 'Los Angeles'],
    state=['California', 'Illinois', 'California', 'Maryland', 'Texas', 'California', 'California'],
    graduation_rate=[0.87, 0.94, 0.78, 0.92, 0.81, 0.83, 0.91 ]
)

nice_weather_cities = bpd.DataFrame().assign(
    city=['La Jolla', 'San Diego', 'Austin', 'Los Angeles'],
    state=['California', 'California', 'Texas', 'California'],
    today_high_temp=['79', '83', '87', '87']
    
)

### Discussion Question

In [ ]:
schools

In [ ]:
nice_weather_cities

**How many rows will be in the output of `nice_weather_cities.merge(schools, on='city')`?**

<center>
<pre>
A. 4      B. 5      C. 6       D. 7       E. 8
</pre>
</center>

### To answer, go to [menti.com](https://menti.com) and enter the code 7374 7077.

### Followup: How many rows will be in the output of `nice_weather_cities.merge(schools, on='state')`?

## Summary

### Summary

- To create groups within a group, we can pass a list to `.groupby`.
    - The result: one row for every unique combination of elements in the specified columns.
- To combine information from multiple DataFrames, we use `.merge`.
    - When using `.merge`, Python searches for a match between a specified column in each DataFrame and combines the rows with a match.
    - No matches = row disappears!
- **Next time**: if-statements and loops.